# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [4]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
#df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [5]:
	
from pyspark.sql import SparkSession
from pyspark.sql import column as col
from pyspark.sql.types import DoubleType
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_imm =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [6]:
#df_immDesc = pd.read_sas('I94_SAS_Labels_Descriptions.SAS', format='SAS7BDAT')

In [7]:
df_imm = df_imm.selectExpr(["cicid AS  id",
                            "i94yr AS  year",
                            "i94mon AS month",
                            "i94res AS origin_country_id",
                            "i94port AS city_id",
                            "arrdate AS arrival_date",
                            "i94mode AS transport_mode",
                            "depdate AS departure_date",
                            "i94visa AS visa_purpose",
                            "visatype AS visa_type",
                            "biryear AS birth_year",
                            "gender",
                            "admnum AS admin_num",
                            "airline AS airline_code",
                            "fltno AS flight_number"])

In [8]:
fname = 'airport-codes_csv.csv'
df_airports = spark.read.csv(fname, header=True)
df_airports = df_airports.selectExpr(["iata_code",
                                      "name",
                                      "type",
                                      "municipality AS city",
                                      "iso_region AS region",
                                      "iso_country AS country",
                                      "continent",
                                      "coordinates",
                                      "gps_code",
                                      "local_code"])

In [9]:
df_join = df_imm.join(df_airports, df_imm.city_id==df_airports.iata_code, 'left')

df_imm = df_join.filter(df_join.iata_code.isNotNull())\
                .select([   "id",
                            "year",
                            "month",
                            "origin_country_id",
                            "city_id",
                            "city",
                            "arrival_date",
                            "transport_mode",
                            "departure_date",
                            "visa_purpose",
                            "visa_type",
                            "birth_year",
                            "gender",
                            "admin_num",
                            "airline_code",
                            "flight_number"  ])

df_airports = df_join.filter(df_join.iata_code.isNotNull())\
                     .select(["iata_code",
                              "name",
                              "type",
                              "city",
                              "region",
                              "country",
                              "continent",
                              "coordinates",
                              "gps_code",
                              "local_code"])\
                     .dropDuplicates()

In [ ]:
#df_airports.show(255)

In [10]:
import pyspark.sql.functions as F

fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = spark.read.csv(fname, header=True)
df_temp = df_temp.filter((df_temp.dt[6:2]=="04") & (df_temp.Country == "United States"))\
                 .groupBy(df_temp.City,df_temp.Latitude,df_temp.Longitude)\
                 .agg(F.avg("AverageTemperature").alias("avg_temp"))\
                 .withColumnRenamed("city", "city2")

df_join = df_imm.join(df_temp, df_imm.city==df_temp.city2, 'left')

df_temp = df_join.selectExpr(["city_id",
                              "city",
                              "Latitude AS latitude",
                              "Longitude AS longitude",
                              "avg_temp"])\
                 .dropDuplicates(subset=['city'])\
                 .filter(df_join.avg_temp.isNotNull())

In [39]:
fname = 'us-cities-demographics.csv'
df_demographics = spark.read.csv(fname, header=True, sep=';')\
                       .withColumnRenamed("City", "city2")\
                       .withColumnRenamed("State", "state")\
                       .withColumnRenamed("State Code", "state_code")\
                       .withColumnRenamed("Median Age", "median_age")\
                       .withColumnRenamed("Male Population", "male_population")\
                       .withColumnRenamed("Female Population", "female_population")\
                       .withColumnRenamed("Total Population", "total_population")\
                       .withColumnRenamed("Number of Veterans", "number_of_veterans")\
                       .withColumnRenamed("Foreign-born", "foreign_born")\
                       .withColumnRenamed("Average Household Size", "household_size")\
                       .withColumnRenamed("Race", "race")

#df_demographics.printSchema()
#df_demographics.show(10)

In [40]:
df_join = df_imm.join(df_demographics, df_imm.city==df_demographics.city2, 'left')
                
df_demographics = df_join.select(["city_id",
                                  "city",
                                  "median_age",
                                  "male_population",
                                  "female_population",
                                  "total_population",
                                  "number_of_veterans",
                                  "foreign_born",
                                  "household_size",
                                  "race"])\
                 .dropDuplicates(subset=['city'])\
                 .filter(df_join.total_population.isNotNull())
df_demographics.show(10)

+-------+-----------+----------+---------------+-----------------+----------------+------------------+------------+--------------+--------------------+
|city_id|       city|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|household_size|                race|
+-------+-----------+----------+---------------+-----------------+----------------+------------------+------------+--------------+--------------------+
|    CHS| Charleston|      35.0|          63956|            71568|          135524|              9368|        5767|           2.4|American Indian a...|
|    SAV|   Savannah|      30.3|          69389|            76295|          145684|              9717|       10355|          2.57|               Asian|
|    OMA|      Omaha|      34.2|         218789|           225098|          443887|             24503|       48263|          2.47|American Indian a...|
|    ANC|  Anchorage|      32.2|         152945|           145750|          298695|     

In [41]:
df_demographics.count()

60

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.